In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
import glob
import os
import pandas as pd
import getpass
import chromadb
from langchain_openai import OpenAIEmbeddings

jangan pake full knowledges pake 1 file aja biar hemat" 😊

In [5]:
def load_knowledges(path='knowledges'):
    docs = list()

    txt_files = glob.glob(f'{path}/*.txt')
    for txt in txt_files:
        loader = TextLoader(txt, autodetect_encoding=True)
        docs.extend(loader.load())
    pdf_files = glob.glob(f'{path}/*.pdf')
    for pdf in pdf_files:
        loader = PyPDFLoader(pdf)
        docs.extend(loader.load())

    return docs
docs = load_knowledges()

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
import uuid

In [12]:
def simple_retriever_setup(docs, add_documents=True, delete_existed_collection=False):
    collection_name = "simple"
    # splitting
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap=500)
    chunks = text_splitter.split_documents(docs)

    # setup chromadb
    client = chromadb.PersistentClient()
    if delete_existed_collection:
        client.delete_collection(collection_name)
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # embeddings model
    embedding_model = OpenAIEmbeddings()

    # add chunks into collection
    if add_documents:
        ids = [str(uuid.uuid1()) for _ in range(len(chunks))]
        documents = [c.page_content for c in chunks]
        embeddings = embedding_model.embed_documents(documents)
        metadatas = [c.metadata for c in chunks]
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas
        )

    docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embedding_model)
    retriever = docsearch.as_retriever(search_kwargs={"k":3}, search_type="similarity")

    return retriever

In [13]:
retriever = simple_retriever_setup(docs)

In [16]:
retriever.invoke('teknologi sains data')

[Document(page_content='b. Menyempurnakan penelitian yang berjalan terus-menerus;\nc. Bersifat\tjujur,\tprofesional, berperikemanusiaan dan memperhatikan faktor-faktor ketepatan, seksama dan cermat, religious, serta tidak mengandung ketimpangan gender;\nd. Memberikan penemuan yang baru;\ne. Bermanfaat\tbagi\tUNAIR\tsecara\tilmiah, institusional, dan finansial.\nf. Berbasis kompetensi logis; dan\ng. Mengingat aspek akuntabilitas.\n\n2. Hubungan Peneliti dengan Mahasiswa\n\nDalam melakukan penelitian, seorang dosen wajib melibatkan mahasiswa sebagai bentuk pemenuhan syarat akademik atau arena pembelajaran, aktualitas kompetensi bidang keilmuan, dan pengembangan pribadi.\n3. Penelitian Dasar dan Terapan\n\nSebagai peneliti, seorang dosen wajib:\na. Mengarahkan penelitian untuk kemajuan ilmu pengetahuan dan/atau perolehan hak paten untuk mendorong perkembangan industri nasional;\nb. Meningkatkan ketahanan nasional melalui penggalian sumber daya alam; dan\nc. Menyinergikan berbagai macam di

In [19]:
from langchain_experimental.text_splitter import SemanticChunker

In [38]:
# Semantic
def semantic_splitting_retriever_setup(docs, add_documents=True, delete_existed_collection=False):
    collection_name="semantic_splitting"

    # splitting
    text_splitter = SemanticChunker(OpenAIEmbeddings())
    chunks = text_splitter.split_documents(docs)

    # setup chromadb
    client = chromadb.PersistentClient()
    if delete_existed_collection:
        client.delete_collection(collection_name)
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # embeddings model
    embedding_model = OpenAIEmbeddings()

    # add chunks into collection
    if add_documents:
        ids = [str(uuid.uuid1()) for _ in range(len(chunks))]
        documents = [c.page_content for c in chunks]
        embeddings = embedding_model.embed_documents(documents)
        metadatas = [c.metadata for c in chunks]    
        collection.add(
            ids=ids,
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas
        )

    docsearch = Chroma(persist_directory='chroma', collection_name=collection_name, embedding_function=embedding_model)
    retriever = docsearch.as_retriever(search_kwargs={"k":3}, search_type="similarity")

    return retriever    

In [40]:
semantic_splitting_retriever = semantic_splitting_retriever_setup(
    docs=docs, 
    delete_existed_collection=True
)

In [41]:
semantic_splitting_retriever.invoke('dosen')

[Document(page_content='Menghargai penemuan dan pendapat akademisi lain; dan\nc. Tidak semata-mata untuk kepentingan pribadi. Kedua, seorang dosen wajib menjunjung kebebasan mimbar akademik, yaitu kebebasan menyampaikan pikiran dan pendapat dalam lingkungan serta forum akademik dalam bentuk ceramah, seminar, dan kegiatan ilmiah lainnya sesuai dengan norma serta kaidah keilmuan. Seorang dosen juga memiliki kewajiban untuk selalu mawas diri dan mengevaluasi kinerja sebagai dosen dalam upayanya membina dan mengembangkan karier akademik dan profesi. Ketiga, seorang dosen wajib menumbuhkembangkan suasana akademik di lingkungan kerjanya. Dalam berkomunikasi, baik secara lisan maupun tertulis, dosen menggunakan bahasa yang sopan dan santun, tidak emosional, berpikir jernih, serta tidak menyinggung perasaan orang lain. Keempat, seorang dosen wajib memelihara dan menumbuhkembangkan masyarakat akademik antardosen, yaitu:\na. Memegang teguh dan menghormati hak dan kebebasan akademik serta hak keb